## Prepare imports

In [7]:
from datasets import get_datasets, synsetid_to_cate
from args import get_parser
from pprint import pprint
# from metrics.evaluation_metrics import EMD_CD
# from metrics.evaluation_metrics import jsd_between_point_cloud_sets as JSD
# from metrics.evaluation_metrics import compute_all_metrics
from collections import defaultdict
from models.networks import PointFlow
import os
import torch
import numpy as np
import torch.nn as nn
import pyvista as pv



################################################################################
# Import model and additional stuff
################################################################################

from omegaconf import OmegaConf
from model_wrapper import TopologicalModelVAE
from load_models import load_encoder, load_vae


################################################################################
################################################################################


def get_test_loader(args):
    _, te_dataset = get_datasets(args)
    if args.resume_dataset_mean is not None and args.resume_dataset_std is not None:
        mean = np.load(args.resume_dataset_mean)
        std = np.load(args.resume_dataset_std)
        te_dataset.renormalize(mean, std)
    loader = torch.utils.data.DataLoader(
        dataset=te_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        drop_last=False,
    )
    return loader



In [8]:
p = get_parser()
args, unknown = p.parse_known_args()



In [9]:
args.dims = '512-512-512'
args.latent_dims = '256-256'
args.resume_checkpoint = 'pretrained_models/gen/airplane/checkpoint.pt'
args.use_latent_flow = True

In [10]:
args

Namespace(input_dim=3, dims='512-512-512', latent_dims='256-256', num_blocks=1, latent_num_blocks=1, layer_type='concatsquash', time_length=0.5, train_T=True, nonlinearity='tanh', use_adjoint=True, solver='dopri5', atol=1e-05, rtol=1e-05, batch_norm=True, sync_bn=False, bn_lag=0, use_latent_flow=True, use_deterministic_encoder=False, zdim=128, optimizer='adam', batch_size=50, lr=0.001, beta1=0.9, beta2=0.999, momentum=0.9, weight_decay=0.0, epochs=100, seed=None, recon_weight=1.0, prior_weight=1.0, entropy_weight=1.0, scheduler='linear', exp_decay=1.0, exp_decay_freq=1, dataset_type='shapenet15k', cates=['airplane'], data_dir='data/ShapeNetCore.v2.PC15k', mn40_data_dir='data/ModelNet40.PC15k', mn10_data_dir='data/ModelNet10.PC15k', dataset_scale=1.0, random_rotate=False, normalize_per_shape=False, normalize_std_per_axis=False, tr_max_sample_points=2048, te_max_sample_points=2048, num_workers=4, log_name=None, viz_freq=10, val_freq=10, log_freq=10, save_freq=10, no_validation=False, sav

In [11]:
model = PointFlow(args)
def _transform_(m):
        return nn.DataParallel(m, device_ids=[0])

model = model.cuda()
model.multi_gpu_wrapper(_transform_)
print("Resume Path:%s" % args.resume_checkpoint)
checkpoint = torch.load(args.resume_checkpoint)
model.load_state_dict(checkpoint)
model.eval()

Number of trainable parameters of Point CNF: 927513
Number of trainable parameters of Latent CNF: 134145
Resume Path:pretrained_models/gen/airplane/checkpoint.pt


PointFlow(
  (encoder): DataParallel(
    (module): Encoder(
      (conv1): Conv1d(3, 128, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
      (conv4): Conv1d(256, 512, kernel_size=(1,), stride=(1,))
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc1_m): Linear(in_features=512, out_features=256, bias=True)
      (fc2_m): Linear(in_features=256, out_features=128, bias=True)
      (fc3_m): Linear(in_features=128, out_features=128, bias=True)
      (fc_bn1_m): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc_

In [12]:
with torch.no_grad():
    loader = get_test_loader(args)
    all_sample = []
    all_ref = []
    for data in loader:
        idx_b, te_pc = data["idx"], data["test_points"]
        te_pc = te_pc.cuda() if args.gpu is None else te_pc.cuda(args.gpu)
        B, N = te_pc.size(0), te_pc.size(1)
        _, out_pc = model.sample(B, N)

        # denormalize
        m, s = data["mean"].float(), data["std"].float()
        m = m.cuda() if args.gpu is None else m.cuda(args.gpu)
        s = s.cuda() if args.gpu is None else s.cuda(args.gpu)
        out_pc = out_pc * s + m
        te_pc = te_pc * s + m

        all_sample.append(out_pc)
        all_ref.append(te_pc)

    sample_pcs = torch.cat(all_sample, dim=0)
    ref_pcs = torch.cat(all_ref, dim=0)

Total number of data:2832
Min number of points: (train)2048 (test)2048
Total number of data:405
Min number of points: (train)2048 (test)2048


In [13]:
jnt = torch.cat([sample_pcs,ref_pcs],axis=1)
jnt.shape

torch.Size([405, 4096, 3])

In [14]:
s = sample_pcs - sample_pcs.mean(dim=-2,keepdim=True)
s /= s.norm(dim=-1,keepdim=True)

In [15]:
s[1].norm(dim=-1)

tensor([1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000], device='cuda:0')

In [16]:
# means = sample_pcs.mean(dim=-2,keepdim=True)
# sample_pcs = sample_pcs - means

In [17]:
# means.squeeze()

In [18]:
sample_pcs.norm(dim=-1).max(dim=-1)[0].max()

tensor(0.6900, device='cuda:0')

In [19]:
pl = pv.Plotter(shape=(3,10), window_size=[1600, 600],border=False,polygon_smoothing=True)
offset = 290
for col in range(10):
    points = ref_pcs[col+offset].reshape(-1, 3).detach().cpu().numpy()
    pl.subplot(0, col)
    actor = pl.add_points(
        points,
        style="points",
        emissive=False,
        show_scalar_bar=False,
        render_points_as_spheres=True,
        scalars=points[:, 2],
        point_size=2,
        ambient=0.2, 
        diffuse=0.8, 
        specular=0.8,
        specular_power=40, 
        smooth_shading=True
    )
    points = sample_pcs[col+offset].reshape(-1, 3).detach().cpu().numpy()
    pl.subplot(1, col)
    actor = pl.add_points(
        points,
        style="points",
        emissive=False,
        show_scalar_bar=False,
        render_points_as_spheres=True,
        scalars=points[:, 2],
        point_size=2,
        ambient=0.2, 
        diffuse=0.8, 
        specular=0.8,
        specular_power=40, 
        smooth_shading=True
    )
    points = jnt[col+offset].reshape(-1, 3).detach().cpu().numpy()
    pl.subplot(2, col)
    actor = pl.add_points(
        points,
        style="points",
        emissive=False,
        show_scalar_bar=False,
        render_points_as_spheres=True,
        scalars=points[:, 2],
        point_size=2,
        ambient=0.2, 
        diffuse=0.8, 
        specular=0.8,
        specular_power=40, 
        smooth_shading=True
    )


pl.background_color = "w"
pl.link_views()
pl.camera_position = "yz"
pos = pl.camera.position
pl.camera.position = (pos[0],pos[1],pos[2]+3)
pl.camera.azimuth = -45
pl.camera.elevation = 10
# create a top down light
light = pv.Light(position=(0, 0, 3), positional=True,
                cone_angle=50, exponent=20, intensity=.2)
pl.add_light(light)
pl.camera.zoom(1.3)
pl.screenshot(f"./pointcloud_{offset}.png",transparent_background=True,scale=2)
pl.show()

Widget(value='<iframe src="http://localhost:60916/index.html?ui=P_0x1f2304891e0_0&reconnect=auto" class="pyvis…

# Reference PointClouds

In [20]:
# sample_pcs = torch.tensor(np.load("./pretrained_models/gen/chair/model_out_smp.npy"))


pl = pv.Plotter(shape=(8,8), window_size=[1600, 1600],border=False,polygon_smoothing=True)
offset = 64
for row in range(8):
    for col in range(8):
        points = sample_pcs[row*8 + col + offset].reshape(-1, 3).detach().cpu().numpy()
        pl.subplot(row, col)
        actor = pl.add_points(
            points,
            style="points",
            emissive=False,
            show_scalar_bar=False,
            render_points_as_spheres=True,
            scalars=points[:, 2],
            point_size=2,
            ambient=0.2, 
            diffuse=0.8, 
            specular=0.8,
            specular_power=40, 
            smooth_shading=True
        )


pl.background_color = "w"
pl.link_views()
pl.camera_position = "yz"
pos = pl.camera.position
pl.camera.position = (pos[0],pos[1],pos[2]+3)
pl.camera.azimuth = -45
pl.camera.elevation = 10
# create a top down light
light = pv.Light(position=(0, 0, 3), positional=True,
                cone_angle=50, exponent=20, intensity=.2)
pl.add_light(light)
pl.camera.zoom(1.3)
pl.show()

Widget(value='<iframe src="http://localhost:60916/index.html?ui=P_0x1f24dba97e0_1&reconnect=auto" class="pyvis…

In [21]:
samples_norm = sample_pcs.norm(dim=-1)
ref_norm = ref_pcs.norm(dim=-1)
print(ref_norm.shape)
print(samples_norm.shape)


print(ref_norm.max(axis=-1)[0])
print(samples_norm.max(axis=-1)[0])

torch.Size([405, 2048])
torch.Size([405, 2048])
tensor([0.4226, 0.4307, 0.4141, 0.4635, 0.4222, 0.4152, 0.4568, 0.4234, 0.4285,
        0.4188, 0.4052, 0.4329, 0.3984, 0.3675, 0.4370, 0.4203, 0.3671, 0.4309,
        0.4007, 0.4282, 0.3954, 0.4156, 0.4053, 0.3767, 0.4292, 0.4333, 0.4382,
        0.3808, 0.4402, 0.4178, 0.4351, 0.4322, 0.3726, 0.4351, 0.3932, 0.3923,
        0.4174, 0.3703, 0.4378, 0.4046, 0.3993, 0.4396, 0.4384, 0.5081, 0.4458,
        0.3946, 0.4226, 0.3814, 0.4206, 0.4359, 0.4360, 0.5293, 0.3934, 0.3668,
        0.4268, 0.4288, 0.3602, 0.4404, 0.3848, 0.4229, 0.4659, 0.4854, 0.4199,
        0.4000, 0.4432, 0.4478, 0.4171, 0.3995, 0.3899, 0.4393, 0.3931, 0.4010,
        0.4284, 0.4056, 0.4093, 0.4217, 0.4116, 0.4237, 0.3915, 0.4350, 0.5025,
        0.3736, 0.4304, 0.4485, 0.3818, 0.4070, 0.3733, 0.3923, 0.4467, 0.4450,
        0.4763, 0.4420, 0.4096, 0.4059, 0.4122, 0.4191, 0.4946, 0.4270, 0.3704,
        0.4129, 0.3798, 0.4175, 0.3869, 0.4071, 0.3900, 0.4651, 0.3956, 

In [22]:
print(samples_norm.max(axis=-1)[0].min())

tensor(0.2452, device='cuda:0')
